# Universal Blog Word Cloud Generator 📝☁️

**使い方:**
1. **ステップ1**でブログのURLと期間を設定します。
2. **ステップ2**を実行して、ブログの全テキストを取得します（この処理は時間がかかります）。
3. **ステップ3**を実行すると、最初の分析結果が表示されます。結果を見ながらテキストボックス内の除外ワードを編集し、何度でもステップ3を再実行して結果を調整できます。

### ステップ1：分析対象の設定

In [ ]:
#@title ◆ ブログ情報と期間を入力 ◆
#@markdown --- 
#@markdown **分析したいブログのURLを入力してください**
blog_url = "" #@param {type:"string"}
#@markdown 
#@markdown --- 
#@markdown **基準日（この日まで）を指定してください。空欄の場合は今日になります。**
base_date_str = "" #@param {type:"date"}
#@markdown **基準日から何日遡って分析するか指定してください。**
days_to_go_back = 30 #@param {type:"slider", min:1, max:1000, step:1}

### ステップ2：ブログ記事の取得（時間のかかる処理）

設定が終わったら、下のセルを実行してブログの情報を取得します。この処理は一度だけ実行してください。

In [ ]:
#@title ▼ 記事の取得を実行
import os
from datetime import datetime, timedelta, timezone
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

print("環境の準備中です...")
!pip install requests beautifulsoup4 janome wordcloud matplotlib > /dev/null 2>&1
font_path = '/content/NotoSansCJKjp-Regular.otf'
if not os.path.exists(font_path) or os.path.getsize(font_path) < 1024*1024:
    !wget -q -O {font_path} https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/Japanese/NotoSansCJKjp-Regular.otf
github_repo_url = "https://github.com/amufaamo/blog-to-tagcloud"
stopwords_path = '/content/stopwords.txt'
stopwords_url = github_repo_url.replace('github.com', 'raw.githubusercontent.com') + '/main/stopwords.txt'
!wget -q -O {stopwords_path} {stopwords_url}
print("準備が完了しました！\n")

PLATFORM_CONFIGS = {
    'blogger': {'post_container': 'article.post-outer-container','permalink': 'h3.post-title a','date': 'time.published','date_attribute': 'datetime','content_body': 'div.post-body','pagination': 'a.blog-pager-older-link'},
    'hatenablog': {'post_container': 'article.entry','permalink': 'h1.entry-title a, h2.entry-title a','date': 'time[datetime]','date_attribute': 'datetime','content_body': 'div.entry-content','pagination': 'a[rel="next"]'},
    'ameblo': {'post_container': 'li[data-unique-entry-id]','permalink': 'a[data-gtm-user-entry-title]','date': 'time','date_attribute': 'datetime','content_body': 'div[data-unique-entry-body]','pagination': 'a[data-gtm-button-name="記事一覧_次へ"]'},
    'note': {'post_container': 'div.o-cardNote','permalink': 'a.o-cardNote__link','date': 'time','date_attribute': 'datetime','content_body': 'div.note-common-styles__p','pagination': None}
}

def detect_platform(url, soup):
    if 'hatenablog' in url or soup.select_one('link[href*="cdn.hatena.com"]'): return 'hatenablog'
    if 'ameblo.jp' in url or soup.select_one('meta[property="og:site_name"][content="Ameba"]'): return 'ameblo'
    if 'note.com' in url: return 'note'
    if 'blogspot.com' in url or soup.select_one('meta[content="blogger"]'): return 'blogger'
    return None

def analyze_blog(base_url, start_date, end_date):
    all_text = ""
    current_url = base_url
    page_num = 1
    print(f"ブログの分析を開始します。対象期間: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
    try:
        response = requests.get(current_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        return None
    platform = detect_platform(base_url, soup)
    if not platform: return None
    print(f"プラットフォーム: {platform.capitalize()} を検出しました。")
    config = PLATFORM_CONFIGS[platform]
    while current_url:
        if page_num > 1:
            try:
                response = requests.get(current_url, headers={'User-Agent': 'Mozilla/5.0'})
                response.encoding = 'utf-8'
                soup = BeautifulSoup(response.text, 'html.parser')
            except:
                break
        time.sleep(1)
        posts = soup.select(config['post_container'])
        if not posts: break
        stop_crawling = False
        for post in posts:
            date_tag = post.select_one(config['date'])
            link_tag = post.select_one(config['permalink'])
            if not date_tag or not link_tag: continue
            post_date_str = date_tag.get(config['date_attribute'])
            if not post_date_str: continue
            try:
                post_date = datetime.fromisoformat(post_date_str.replace('Z', '+00:00'))
            except ValueError:
                continue
            if start_date <= post_date <= end_date:
                post_url = urljoin(base_url, link_tag['href'])
                try:
                    post_res = requests.get(post_url, headers={'User-Agent': 'Mozilla/5.0'})
                    post_res.encoding = 'utf-8'
                    post_soup = BeautifulSoup(post_res.text, 'html.parser')
                    content_body = post_soup.select_one(config['content_body'])
                    if content_body:
                        all_text += content_body.get_text() + "\n"
                    time.sleep(1)
                except:
                    pass
            elif post_date < start_date:
                stop_crawling = True
                break
        if stop_crawling:
            break
        if config['pagination']:
            next_page_tag = soup.select_one(config['pagination'])
            if next_page_tag and next_page_tag.has_attr('href'):
                current_url = urljoin(base_url, next_page_tag['href'])
                page_num += 1
            else:
                current_url = None
        else:
            current_url = None
    return all_text

# メイン処理
if not blog_url: raise ValueError("ステップ1でブログのURLが入力されていません。")
jst = timezone(timedelta(hours=9))
today = datetime.now(jst)
if base_date_str: end_date_obj = datetime.strptime(base_date_str, '%Y-%m-%d').replace(hour=23, minute=59, second=59, tzinfo=jst)
else: end_date_obj = today
start_date_obj = end_date_obj - timedelta(days=days_to_go_back)

scraped_text = analyze_blog(blog_url, start_date_obj, end_date_obj)
if scraped_text:
    print("\n✅ ブログ記事の取得が完了しました。ステップ3に進んで分析を実行してください。")
else:
    print("\nエラー: 分析対象の記事が見つかりませんでした。")

### ステップ3：分析と対話的な再分析

下のテキストボックスに、`stopwords.txt`から読み込んだ除外ワードが表示されます。リストを編集して**このセルを実行（▶）**すると、即座に結果が更新されます。

In [ ]:
#@title ▼ 除外ワードを編集して分析・再分析
from janome.tokenizer import Tokenizer
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import os
import re

# ステップ2が実行済みかチェック
if 'scraped_text' not in globals() or not scraped_text:
    print("エラー：先にステップ2を実行して、ブログのテキストを取得してください。")
else:
    # --- 除外ワードリストの準備 ---
    # 初回実行時のみファイルから読み込み、テキストエリアのデフォルト値として利用します。
    # 2回目以降の実行では、ユーザーが編集した内容がこのテキストエリアに維持されます。
    try:
        # このセルが初めて実行される時にだけ、ファイルから読み込む
        if 'editable_stopwords' not in globals():
            with open('/content/stopwords.txt', 'r', encoding='utf-8') as f:
                # editable_stopwords にファイル内容を保存
                editable_stopwords = f.read()
                print("stopwords.txtから除外ワードを読み込みました。")
    except FileNotFoundError:
        # ファイルが見つからない場合（ステップ2が未実行など）
        if 'editable_stopwords' not in globals():
            editable_stopwords = "" # 空の状態で開始
            print("警告: stopwords.txt が見つかりませんでした。")

    #@markdown ---
    #@markdown **↓のリストを自由に編集し、このセル（▶）を再実行すると、結果が更新されます。**
    #@markdown （1行に1単語の形式で追加・削除してください）
    editable_stopwords = editable_stopwords #@param {type:"string"}

    # --- テキストの分析 ---
    def reanalyze_text(text, stopwords_str):
        # テキストエリアの文字列を改行で分割し、除外ワードのセットを作成
        stopwords = {line.strip() for line in stopwords_str.splitlines() if line.strip()}
        print(f"\n{len(stopwords)}個の除外ワードでテキストを再分析中...")
        t = Tokenizer()
        words = [token.surface for token in t.tokenize(text)
                 if token.surface not in stopwords and
                 token.part_of_speech.startswith(('名詞', '動詞', '形容詞')) and
                 len(token.surface) > 1 and not re.match(r'^[0-9a-zA-Z]+$', token.surface)]
        return Counter(words)

    # --- 結果の表示 ---
    def show_results(counter):
        print("\n--- ✅単語の頻度ランキング TOP 50 ---")
        for word, count in counter.most_common(50):
            print(f"{word}: {count}回")

        font_path = '/content/NotoSansCJKjp-Regular.otf'
        # ステップ2でダウンロードしたフォントファイルの存在をチェック
        if os.path.exists(font_path) and os.path.getsize(font_path) > 1000000:
          print("\nワードクラウド画像を生成中...")
          try:
            wordcloud = WordCloud(width=1200, height=600, background_color='white', font_path=font_path, max_words=150, colormap='viridis').generate_from_frequencies(counter)
            plt.figure(figsize=(15, 8))
            plt.imshow(wordcloud, interpolation='bilinear')
            plt.axis("off")
            plt.show()
          except Exception as e:
            # ワードクラウド生成中にエラーが起きた場合
            print(f"\nエラー: ワードクラウドの生成に失敗しました。詳細: {e}")
        else:
            # フォントファイルが見つからない場合
            print("\nエラー：ワードクラウド生成用のフォントファイルが見つかりません。ステップ2を再実行してください。")

    # --- メイン処理 ---
    # 編集された除外ワードを使って分析を実行
    word_counter = reanalyze_text(scraped_text, editable_stopwords)
    if word_counter:
        show_results(word_counter)
    else:
        print("\n分析の結果、有効な単語が見つかりませんでした。除外ワードリストを見直してみてください。")